# Sentinel-2 - Near Real-Time Product

The Sentinel-2 near real-time product is an analysis-ready, optical data set, available over Australia for the last 90 days. This notebook demonstrates how to load and display the Sentinel-2 near real-time (NRT) product for [central Canberra](https://www.google.com/maps/place/35%C2%B017'36.4%22S+149%C2%B007'30.7%22E/@-35.2940536,149.1073962,14.28z/data=!4m14!1m7!3m6!1s0x6b164d69b05c9021:0x500ea6ea7695660!2sCanberra+ACT+2601!3b1!8m2!3d-35.2809368!4d149.1300092!3m5!1s0x0:0x0!7e2!8m2!3d-35.2934324!4d149.1251798).

## Step 1
Import the necessary Python packages and initialise the datacube instance as `dc`.

In [ ]:
import datacube
import matplotlib.pyplot as plt
import xarray as xr
import numpy as np
import datetime as dt

dc = datacube.Datacube(app='sentinel2-nrt')

## Step 2
Set the latitude, longitude and time bounds for the analysis. Bounds are given as `(min, max)`. Since the Sentinel-2 NRT product covers the last 90 days, this example loads one month of data from today.

In [ ]:
canberra_latitude = (-35.25, -35.35)
canberra_longitude = (149.05, 149.15)

end_date = dt.date.today()
start_date = end_date - dt.timedelta(days=30)
canberra_time = (start_date, end_date)

## Step 3
Load the dataset for the specified bounds through the `dc.load()` command. The dataset to be loaded is given as the product. In this case, the two products are `s2a_nrt_granule` and `s2b_nrt_granule`, corresponding to the two [Sentinel-2 satellites](https://sentinel.esa.int/web/sentinel/missions/sentinel-2). After loading both products, they are combined into a single dataset called `sentinel2_nrt`. The measurements loaded for each product are the red, green and blue spectral bands (`nbar_red`, `nbar_green`, `nbar_blue`), and the pixel quality information `fmask`.  

The data will take some time to load, indicated by the `In [*]` symbol to the left of the code cell. The load is finished when the `*` is replaced by a number. 

In [ ]:
measurement_list = ['nbar_red', 'nbar_green', 'nbar_blue', 'fmask']

sentinel2a = dc.load(
    product='s2a_nrt_granule',
    group_by='solar_day',
    measurements=measurement_list,
    latitude=canberra_latitude,
    longitude=canberra_longitude,
    time=canberra_time,
    resolution=(-10,10),
    output_crs='epsg:3577'
)

sentinel2b = dc.load(
    product='s2b_nrt_granule',
    group_by='solar_day',
    latitude=canberra_latitude,
    longitude=canberra_longitude,
    time=canberra_time,
    resolution=(-10,10),
    output_crs='epsg:3577'
)

sentinel2_nrt = xr.concat([sentinel2a, sentinel2b], dim='time')

## Step 4
View the contents and properties of the loaded data by printing the `sentinel2_nrt` dataset. From the print out, note that the dataset has three dimensions (`time`, `x`, `y`) and data variables corresponding to the `measurement_list` provided in the previous cell. The `Dimensions` information reveals the number of entries for each dimension.

In [ ]:
print(sentinel2_nrt)

## Step 5

Plot the true-colour image by combining the red (`nbar_red`), green (`nbar_green`) and blue (`nbar_blue`) measurements at a given time. Change the `timestep` variable to view the true-colour image on different dates.

In [ ]:
timestep = 0
bands = ['nbar_red', 'nbar_green', 'nbar_blue']

sentinel2_nrt[bands].isel(time=timestep).to_array().plot.imshow(robust=True, figsize=(8, 8))
plt.show()

## Step 6

Calculate a mask layer to only select pixels that are valid. The pixel quality measurement for the Sentinel-2 near real-time product is stored in the `fmask` measurement. The values are described in the table below. In the following cell, the mask is constructed from all pixels that are classified as Null, Cloud or Cloud shadow, and inverted to track only clear pixels. The mask is then applied to the `sentinel2_nrt` data set to produce the `clear_sentinel2_nrt` data set, which can then be plotted.

| `fmask` value | description   |
|------|------|
|   0  |Null |
|   1  |Valid |
|   2  |Cloud |
|   3  |Cloud shadow |
|   4  |Snow |
|   5  |Water |

In [ ]:
clear_pixels = np.isin(sentinel2_nrt.fmask, test_elements=[0, 2, 3], invert=True)
sentinel2_nrt_clear = sentinel2_nrt.where(clear_pixels)

sentinel2_nrt_clear[bands].isel(time=timestep).to_array().plot.imshow(robust=True, figsize=(8, 8))
plt.show()